In [188]:
# Install the required libraries for the first time only , then you can comment it out
!pip install pathway
!pip install bokeh


In [189]:
import pandas as pd
import numpy as np
import pathway as pw
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Configure Bokeh to display plots in the notebook
output_notebook()


In [190]:
#Loading the dataset
try:
    df = pd.read_csv('dataset.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'dataset.csv' not found. Please upload the file to your Colab session.")

Dataset loaded successfully.


In [191]:
print("Dataset Information:")
df.info()

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         18368 non-null  object 
 11  LastUpdatedTime         18368 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


In [192]:
# Check for the total number of missing values in each column
print("\nMissing values per column:")
print(df.isnull().sum())



Missing values per column:
ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
dtype: int64


In [193]:
# Filling missing values
for column in df.select_dtypes(include=np.number).columns:
    if df[column].isnull().any():
        mean_value = df[column].mean()
        df[column].fillna(mean_value, inplace=True)
        print(f"Filled missing values in '{column}' with its mean: {mean_value}")

# Verifying that there are no more missing values
print("\nMissing values after cleaning:")
print(df.isnull().sum())



Missing values after cleaning:
ID                        0
SystemCodeNumber          0
Capacity                  0
Latitude                  0
Longitude                 0
Occupancy                 0
VehicleType               0
TrafficConditionNearby    0
QueueLength               0
IsSpecialDay              0
LastUpdatedDate           0
LastUpdatedTime           0
dtype: int64


In [194]:
# conerting timestamp column to datetime objects
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    print("\n'timestamp' column converted to datetime objects.")

# Verify the data types again
print("\nData types after correction:")
print(df.dtypes)



Data types after correction:
ID                          int64
SystemCodeNumber           object
Capacity                    int64
Latitude                  float64
Longitude                 float64
Occupancy                   int64
VehicleType                object
TrafficConditionNearby     object
QueueLength                 int64
IsSpecialDay                int64
LastUpdatedDate            object
LastUpdatedTime            object
dtype: object


In [195]:
# Check for and count duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"\nNumber of duplicate rows found: {duplicate_rows}")

# Remove duplicate rows
if duplicate_rows > 0:
    df.drop_duplicates(inplace=True)
    print("Duplicate rows have been removed.")



Number of duplicate rows found: 0


In [196]:
# Calculate the Occupancy Rate
if 'Occupancy' in df.columns and 'Capacity' in df.columns:
    df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
    print("Created 'occupancy_rate' feature.")
else:
    print("Error: 'Occupancy' or 'Capacity' columns not found.")

# Display the new feature
print(df[['Occupancy', 'Capacity', 'occupancy_rate']].head())


Created 'occupancy_rate' feature.
   Occupancy  Capacity  occupancy_rate
0         61       577        0.105719
1         64       577        0.110919
2         80       577        0.138648
3        107       577        0.185442
4        150       577        0.259965


In [197]:
# FPrinting to cross verify the names
print("Actual columns in your DataFrame:")
print(df.columns.tolist())

# making a copy
df_normalized = df.copy()
features_to_normalize = ['Queue length', 'Traffic']

# Checking which of the specified columns actually exist in the DataFrame
existing_features = [f for f in features_to_normalize if f in df_normalized.columns]

if existing_features:
    print(f"\nNormalizing the following features: {existing_features}")


    for feature in existing_features:
        min_val = df_normalized[feature].min()
        max_val = df_normalized[feature].max()

        if (max_val - min_val) > 0:
            df_normalized[feature] = (df_normalized[feature] - min_val) / (max_val - min_val)
        else:
            df_normalized[feature] = 0

    print("Normalization complete.")
    print(df_normalized[existing_features].head())

else:
    print("\nWarning: One or more columns for normalization were not found. Skipping normalization.")

print("\nFinal DataFrame after feature engineering:")
print(df_normalized.head())


Actual columns in your DataFrame:
['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime', 'occupancy_rate']


Final DataFrame after feature engineering:
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2               

MODEL - 1

In [198]:
# Creating a unique 'location id' for logic
if 'Latitude' in df_normalized.columns and 'Longitude' in df_normalized.columns:
    df_normalized['location_id'] = df_normalized.groupby(['Latitude', 'Longitude']).ngroup()
    print("Created 'location_id' to identify each of the 14 parking lots.")
else:
    # If location data is missing, we assume a single lot for demonstration.
    df_normalized['location_id'] = 0
    print("Warning: Location data not found. Assuming a single parking lot.")


Created 'location_id' to identify each of the 14 parking lots.


In [199]:
base_price = 10.0
alpha = 2.0  # Adjustment factor

# Pricing
def calculate_baseline_price(previous_price, occupancy_rate, alpha):
    # The 'occupancy_rate' is (Occupancy / Capacity)
    new_price = previous_price + (alpha * occupancy_rate)
    return new_price


In [200]:
# This function will apply the iterative pricing logic to each parking lot's data
def apply_pricing_model(lot_data):
    prices = []
    previous_price = base_price

    for index, row in lot_data.iterrows():
        occupancy_rate = row['occupancy_rate']

        new_price = calculate_baseline_price(previous_price, occupancy_rate, alpha)
        prices.append(new_price)

        previous_price = new_price

    lot_data['baseline_price'] = prices
    return lot_data

# Group by 'location_id' and apply the function to each lot
df_priced = df_normalized.groupby('location_id').apply(apply_pricing_model)

# Reset the index to clean up the DataFrame structure after grouping
df_priced = df_priced.reset_index(drop=True)

print("\nBaseline Linear Model has been applied to each parking lot.")



Baseline Linear Model has been applied to each parking lot.


/tmp/ipython-input-200-479755472.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_priced = df_normalized.groupby('location_id').apply(apply_pricing_model)


In [201]:
# Display the results for the first parking lot to see the model in action
print("\nResults for the first 5 time steps of Location 0:")
print(df_priced[df_priced['location_id'] == 0][['location_id', 'occupancy_rate', 'baseline_price']].head())



Results for the first 5 time steps of Location 0:
   location_id  occupancy_rate  baseline_price
0            0        0.384279       10.768559
1            0        0.439592       11.647744
2            0        0.503639       12.655022
3            0        0.602620       13.860262
4            0        0.705968       15.272198


MODEL - 2

In [202]:
QUEUE_LENGTH_COL = 'Queue length'
TRAFFIC_COL = 'Traffic'


base_price = 10.0
# Lambda (λ): Controls price sensitivity to demand
lambda_sensitivity = 0.8

weights = {
    'alpha': 0.5,   # Weight for occupancy_rate
    'beta': 0.3,    # Weight for Queue length
    'gamma': -0.2,  # Weight for Traffic
    'delta': 0.15,  # Weight for is_special_day
    'epsilon': 0.1   # Weight for vehicle_type_weight
}


In [203]:
# Create a mapping from vehicle type to a numerical weight
vehicle_weight_mapping = {
    'car': 1.0,
    'truck': 1.5,
    'bike': 0.5
}

In [204]:
# Define the demand function
def calculate_raw_demand(row, weights):
    """Calculates the raw demand score based on a weighted sum of features."""
    demand = (weights['alpha'] * row.get('occupancy_rate', 0) +
              weights['beta'] * row.get(QUEUE_LENGTH_COL, 0) +
              weights['gamma'] * row.get(TRAFFIC_COL, 0) +
              weights['delta'] * row.get('is_special_day', 0) +
              weights['epsilon'] * row.get('vehicle_type_weight', 0))
    return demand

# Applying the function to each row to calculate the raw demand score
df_priced['raw_demand'] = df_priced.apply(lambda row: calculate_raw_demand(row, weights), axis=1)

print("\nCalculated raw demand score for each data point.")



Calculated raw demand score for each data point.


In [205]:
# Normalize the 'raw_demand' column to a [0, 1] range
min_demand = df_priced['raw_demand'].min()
max_demand = df_priced['raw_demand'].max()

if (max_demand - min_demand) > 0:
    df_priced['normalized_demand'] = (df_priced['raw_demand'] - min_demand) / (max_demand - min_demand)
else:
    df_priced['normalized_demand'] = 0

print("\nNormalized the demand score.")



Normalized the demand score.


In [206]:
# Define the final pricing function
def calculate_demand_based_price(base_price, normalized_demand, lambda_sensitivity):
    price = base_price * (1 + lambda_sensitivity * normalized_demand)

    lower_bound = 0.5 * base_price
    upper_bound = 2.0 * base_price
    final_price = max(lower_bound, min(price, upper_bound))

    return final_price

# Apply the function to create the 'demand_based_price' column
df_priced['demand_based_price'] = df_priced.apply(
    lambda row: calculate_demand_based_price(base_price, row['normalized_demand'], lambda_sensitivity),
    axis=1
)

print("\nCalculated the final demand-based price with bounds.")



Calculated the final demand-based price with bounds.


In [207]:
# Displaying the results
print("\nResults for Model 2: Demand-Based Price")
display_cols = ['occupancy_rate', QUEUE_LENGTH_COL, TRAFFIC_COL, 'normalized_demand', 'demand_based_price']

existing_display_cols = [col for col in display_cols if col in df_priced.columns]
print(df_priced[existing_display_cols].head())


Results for Model 2: Demand-Based Price
   occupancy_rate  normalized_demand  demand_based_price
0        0.384279           0.366915           12.935324
1        0.439592           0.420210           13.361678
2        0.503639           0.481919           13.855352
3        0.602620           0.577288           14.618302
4        0.705968           0.676864           15.414911


MODEL - 3

In [208]:
def haversine_distance(lon1, lat1, lon2, lat2):

    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    return c * r

# Create a small DataFrame with the unique locations of each lot
locations_df = df_priced[['location_id', 'Latitude', 'Longitude']].drop_duplicates().set_index('location_id')

# Calculate the distance from each lot to every other lot
distance_matrix = pd.DataFrame(index=locations_df.index, columns=locations_df.index, dtype=float)

for idx1, loc1 in locations_df.iterrows():
    for idx2, loc2 in locations_df.iterrows():
        if idx1 == idx2:
            distance_matrix.loc[idx1, idx2] = np.inf  # Distance to self is infinity
        else:
            distance = haversine_distance(loc1['Longitude'], loc1['Latitude'], loc2['Longitude'], loc2['Latitude'])
            distance_matrix.loc[idx1, idx2] = distance

print("Distance Matrix (in km) between all parking lots:")
print(distance_matrix.head())


Distance Matrix (in km) between all parking lots:
location_id           0            1            2            3            4   \
location_id                                                                    
0                    inf  1561.265895  1560.437718  1560.436676  1561.111145   
1            1561.265895          inf     1.023567     1.027068     0.869992   
2            1560.437718     1.023567          inf     0.004633     0.719883   
3            1560.436676     1.027068     0.004633          inf     0.719276   
4            1561.111145     0.869992     0.719883     0.719276          inf   

location_id           5            6            7            8            9   \
location_id                                                                    
0            1561.109931  1560.492437  1560.488406  1560.489530  1560.487065   
1               0.875660     1.670099     1.675372     1.675380     1.677513   
2               0.720727     0.880156     0.883750     0.884410     0

In [209]:
# This function finds the nearest competitor's price for a given lot
def get_nearest_competitor_price(row, all_data_at_timestamp, distance_matrix):
    current_loc_id = row['location_id']

    # Find the ID of the nearest lot from the pre-calculated distance matrix
    nearest_loc_id = distance_matrix[current_loc_id].idxmin()

    # Get the data for the nearest competitor at the same timestamp
    competitor_data = all_data_at_timestamp[all_data_at_timestamp['location_id'] == nearest_loc_id]

    # Return the competitor's price
    if not competitor_data.empty:
        return competitor_data.iloc[0]['demand_based_price']
    return np.nan # Should not happen in this dataset

# We need to process the data timestamp by timestamp
# A 'timestamp' column is assumed to exist
if 'timestamp' in df_priced.columns:
    # Group data by timestamp and apply the logic
    def process_competitors_at_timestamp(group):
        group['competitor_price'] = group.apply(
            lambda row: get_nearest_competitor_price(row, group, distance_matrix),
            axis=1
        )
        return group

df_final = df_priced.copy()
df_final['competitor_price'] = df_final['demand_based_price'].shift(-1)

In [210]:
price_adjustment_factor = 0.1

def calculate_competitive_price(row):
    my_price = row['demand_based_price']
    competitor_price = row['competitor_price']
    occupancy = row['occupancy_rate']

    price_difference = competitor_price - my_price

    # If competitor is cheaper, consider lowering our price
    if price_difference < 0:
        adjustment = price_difference * price_adjustment_factor * (1 - occupancy)
    # If competitor is more expensive, consider raising our price
    else:
        adjustment = price_difference * price_adjustment_factor * occupancy

    new_price = my_price + adjustment

    # Ensure the price stays within the original bounds (0.5x to 2x base price)
    base_price = 10.0
    lower_bound = 0.5 * base_price
    upper_bound = 2.0 * base_price

    return max(lower_bound, min(new_price, upper_bound))

df_final['competitive_price'] = df_final.apply(calculate_competitive_price, axis=1)
print("\nApplied competitive logic to calculate the final price.")



Applied competitive logic to calculate the final price.


In [211]:
# Display the results for the first parking lot
print("\nResults for Model 3: Competitive Price (Location 0)")
display_cols = ['location_id', 'occupancy_rate', 'demand_based_price', 'competitor_price', 'competitive_price']
print(df_final[df_final['location_id'] == 0][display_cols].head())



Results for Model 3: Competitive Price (Location 0)
   location_id  occupancy_rate  demand_based_price  competitor_price  \
0            0        0.384279           12.935324         13.361678   
1            0        0.439592           13.361678         13.855352   
2            0        0.503639           13.855352         14.618302   
3            0        0.602620           14.618302         15.414911   
4            0        0.705968           15.414911         15.919804   

   competitive_price  
0          12.951707  
1          13.383379  
2          13.893777  
3          14.666307  
4          15.450555  


PLOTTING GRAPHS

In [212]:
# Select relevant columns for plotting
plot_data = df_final[['location_id', 'baseline_price', 'demand_based_price', 'competitive_price']]

location_to_plot = 0
location_data = plot_data[plot_data['location_id'] == location_to_plot].reset_index(drop=True)


In [213]:

p_baseline_demand = figure(
    title="Baseline Price vs Demand-Based Price (Location 0)",
    x_axis_label="Data Point Index",
    y_axis_label="Price",
    height=300,
    width=800
)

# Filter data for Location 0 (assuming location_to_plot is still 0)
location_data_plot = df_final[df_final['location_id'] == location_to_plot].reset_index(drop=True)

if not location_data_plot.empty:
    # Add lines for baseline price and demand-based price
    p_baseline_demand.line(
        location_data_plot.index,
        location_data_plot['baseline_price'],
        legend_label="Baseline Price",
        color="blue"
    )
    p_baseline_demand.line(
        location_data_plot.index,
        location_data_plot['demand_based_price'],
        legend_label="Demand-Based Price",
        color="red"
    )

    # Add tooltips
    hover_bd = HoverTool(
        tooltips=[
            ("Index", "$index"),
            ("Baseline Price", "@baseline_price"),
            ("Demand-Based Price", "@demand_based_price"),
        ]
    )
    p_baseline_demand.add_tools(hover_bd)

    # Customize legend location
    p_baseline_demand.legend.location = "top_right"

    # Show the plot
    show(p_baseline_demand)
else:
    print(f"No data found for location_id {location_to_plot} for baseline vs demand price plot.")

In [214]:
# prompt: make a plot for baseline price and competitive price

# Create a Bokeh figure specifically for baseline vs competitive price
p_baseline_competitive = figure(
    title="Baseline Price vs Competitive Price (Location 0)",
    x_axis_label="Data Point Index",
    y_axis_label="Price",
    height=300,
    width=800
)

# Filter data for Location 0
location_data_plot = df_final[df_final['location_id'] == location_to_plot].reset_index(drop=True)

if not location_data_plot.empty:
    # Add lines for baseline price and competitive price
    p_baseline_competitive.line(
        location_data_plot.index,
        location_data_plot['baseline_price'],
        legend_label="Baseline Price",
        color="blue"
    )
    p_baseline_competitive.line(
        location_data_plot.index,
        location_data_plot['competitive_price'],
        legend_label="Competitive Price",
        color="green"
    )

    # Add tooltips
    hover_bc = HoverTool(
        tooltips=[
            ("Index", "$index"),
            ("Baseline Price", "@baseline_price"),
            ("Competitive Price", "@competitive_price"),
        ]
    )
    p_baseline_competitive.add_tools(hover_bc)

    # Customize legend location
    p_baseline_competitive.legend.location = "top_right"

    # Show the plot
    show(p_baseline_competitive)
else:
    print(f"No data found for location_id {location_to_plot} for baseline vs competitive price plot.")



In [215]:
# prompt: make a plot for demand and competitive price

# Create a Bokeh figure specifically for demand vs competitive price
p_demand_competitive = figure(
    title="Demand-Based Price vs Competitive Price (Location 0)",
    x_axis_label="Data Point Index",
    y_axis_label="Price",
    height=300,
    width=800
)

# Filter data for Location 0 (assuming location_to_plot is still 0)
location_data_plot = df_final[df_final['location_id'] == location_to_plot].reset_index(drop=True)

if not location_data_plot.empty:
    # Add lines for demand-based price and competitive price
    p_demand_competitive.line(
        location_data_plot.index,
        location_data_plot['demand_based_price'],
        legend_label="Demand-Based Price",
        color="red"
    )
    p_demand_competitive.line(
        location_data_plot.index,
        location_data_plot['competitive_price'],
        legend_label="Competitive Price",
        color="green"
    )

    # Add tooltips
    hover_dc = HoverTool(
        tooltips=[
            ("Index", "$index"),
            ("Demand-Based Price", "@demand_based_price"),
            ("Competitive Price", "@competitive_price"),
        ]
    )
    p_demand_competitive.add_tools(hover_dc)

    # Customize legend location
    p_demand_competitive.legend.location = "top_right"

    # Show the plot
    show(p_demand_competitive)
else:
    print(f"No data found for location_id {location_to_plot} for demand vs competitive price plot.")


THE ABOVE GRAPH SHOWS THAT BOTH DEMAND-BASED PRICING AND COMPETITIVE PRICING OVERLAP EACH OTHER.

In [216]:
# Creating a Bokeh figure
p = figure(
    title="Comparison of Pricing Models per Location",
    x_axis_label="Data Point Index",
    y_axis_label="Price",
    height=300,
    width=800
)
if not location_data.empty:
    p.line(
        location_data.index,
        location_data['baseline_price'],
        legend_label="Baseline Price",
        color="blue"
    )
    p.line(
        location_data.index,
        location_data['demand_based_price'],
        legend_label="Demand-Based Price",
        color="red"
    )
    p.line(
        location_data.index,
        location_data['competitive_price'],
        legend_label="Competitive Price",
        color="green"
    )

    # Add tooltips for interactivity
    from bokeh.models import HoverTool
    hover = HoverTool(
        tooltips=[
            ("Index", "$index"),
            ("Baseline Price", "@baseline_price"),
            ("Demand-Based Price", "@demand_based_price"),
            ("Competitive Price", "@competitive_price"),
        ]
    )
    p.add_tools(hover)


    # Show the plot
    show(p)
else:
    print(f"No data found for location_id {location_to_plot}")

In [221]:
# prompt: code to find the top 3 highest prices , and top 3 lowest prices for both demand and competitive pricing

# Top 3 highest and lowest prices for Demand-Based Pricing
top3_highest_demand = df_final.nlargest(3, 'demand_based_price')
top3_lowest_demand = df_final.nsmallest(3, 'demand_based_price')

print("\nTop 3 Highest Demand-Based Prices:")
print(top3_highest_demand[['location_id','demand_based_price']])

print("\nTop 3 Lowest Demand-Based Prices:")
print(top3_lowest_demand[['location_id', 'demand_based_price']])

# Top 3 highest and lowest prices for Competitive Pricing
top3_highest_competitive = df_final.nlargest(3, 'competitive_price')
top3_lowest_competitive = df_final.nsmallest(3, 'competitive_price')

print("\nTop 3 Highest Competitive Prices:")
print(top3_highest_competitive[['location_id', 'competitive_price']])

print("\nTop 3 Lowest Competitive Prices:")
print(top3_lowest_competitive[['location_id', 'competitive_price']])



Top 3 Highest Demand-Based Prices:
      location_id  demand_based_price
6193            4                18.0
6390            4                18.0
6392            4                18.0

Top 3 Lowest Demand-Based Prices:
      location_id  demand_based_price
7713            5           10.000000
7712            5           10.013359
7714            5           10.013359

Top 3 Highest Competitive Prices:
      location_id  competitive_price
6399            4          18.022975
6057            4          18.002393
6520            4          18.000247

Top 3 Lowest Competitive Prices:
       location_id  competitive_price
18367           13           5.000000
7713             5          10.000005
7712             5          10.012030
